In [9]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNet
import cv2

import pathlib

Main_image = "K:/LEGOs/Bricks/BRICKS_COLOR" # zmienić !!!!!!!!!!!!!!!!!!!!!!!!
photo_path_gray = "K:/LEGOs/Bricks/BRICKS_GRAY" # zmienić !!!!!!!!!!!!!!!!!!!!!!!!






data_dir_Brick = Main_image
data_dir_Gray = photo_path_gray

batch_size = 32
img_height = 224
img_width = 224

print("done")

done


In [10]:
train_ds_Brick = tf.keras.utils.image_dataset_from_directory(
  data_dir_Brick,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds_Brick = tf.keras.utils.image_dataset_from_directory(
  data_dir_Brick,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names_Brick = train_ds_Brick.class_names
print(class_names_Brick)
#----------------------------------------------------------------#
train_ds_Gray = tf.keras.utils.image_dataset_from_directory(
  data_dir_Gray,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds_Gray = tf.keras.utils.image_dataset_from_directory(
  data_dir_Gray,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names_Gray = train_ds_Gray.class_names
print(class_names_Gray)
print("done")

Found 6400 files belonging to 5 classes.
Using 5120 files for training.
Found 6400 files belonging to 5 classes.
Using 1280 files for validation.
['Black_Brick', 'Blue_Brick', 'Orange_Brick', 'Red_Brick', 'Yellow_Brick']
Found 6400 files belonging to 5 classes.
Using 5120 files for training.
Found 6400 files belonging to 5 classes.
Using 1280 files for validation.
['2x1', '4x2', '6x1', 'Cylinder', 'Ring']
done


In [11]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds_Brick = train_ds_Brick.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds_Brick = val_ds_Brick.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer_Brick = layers.Rescaling(1./255)

normalized_ds_Brick = train_ds_Brick.map(lambda x, y: (normalization_layer_Brick(x), y))
image_batch_Brick, labels_batch_Brick = next(iter(normalized_ds_Brick))
first_image_Brick = image_batch_Brick[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image_Brick), np.max(first_image_Brick))
#----------------------------------------------------------------#
train_ds_Gray = train_ds_Gray.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds_Gray = val_ds_Gray.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer_Gray = layers.Rescaling(1./255)

normalized_ds_Gray = train_ds_Gray.map(lambda x, y: (normalization_layer_Gray(x), y))
image_batch_Gray, labels_batch_Gray = next(iter(normalized_ds_Gray))
first_image_Gray = image_batch_Gray[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image_Gray), np.max(first_image_Gray))

print("done")


0.0 0.9603495
0.22763109 0.8163168
done


In [12]:
num_classes_Brick = len(class_names_Brick)

input_shape_Brick = (224,224,3)

base_model_Brick = MobileNet(input_shape_Brick,include_top=False, weights='imagenet')

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

base_model_Brick.trainable = False

inputs_Brick = tf.keras.Input(shape = input_shape_Brick)
x_Brick = preprocess_input(inputs_Brick)
x_Brick = layers.Normalization()(x_Brick)
print(x_Brick)
#x = layers.Rescaling(1./255, input_shape=(img_height, img_width, 3))
x = base_model_Brick(x_Brick, training = False)
print(x_Brick)
#----------------------------------------------------------------#
num_classes_Gray = len(class_names_Gray)

input_shape_Gray = (224,224,3)

base_model_Gray = MobileNet(input_shape_Gray,include_top=False, weights='imagenet')

preprocess_input_Gray = tf.keras.applications.mobilenet_v2.preprocess_input

base_model_Gray.trainable = False

inputs_Gray = tf.keras.Input(shape = input_shape_Gray)
x_Gray = preprocess_input_Gray(inputs_Gray)
x_Gray = layers.Normalization()(x_Gray)
print(x_Gray)
#x = layers.Rescaling(1./255, input_shape=(img_height, img_width, 3))
x_Gray = base_model_Gray(x_Gray, training = False)
print(x_Gray)
print("done")

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), name='normalization/truediv:0', description="created by layer 'normalization'")
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), name='normalization/truediv:0', description="created by layer 'normalization'")
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), name='normalization_1/truediv:0', description="created by layer 'normalization_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 1024), dtype=tf.float32, name=None), name='mobilenet_1.00_224/conv_pw_13_relu/Relu6:0', description="created by layer 'mobilenet_1.00_224'")
done


In [13]:
x_Brick = layers.Conv2D(16, 3, padding='same', activation='relu')(x_Brick)
x_Brick = layers.Dropout(0.2)(x_Brick) # for increasing accuracy !!!! <== Important!!!!

#x = layers.GlobalAveragePooling2D()(x)
print(x_Brick)
x_Brick = layers.Conv2D(32, 3, padding='same', activation='relu')(x_Brick)
x_Brick = layers.Dropout(0.2)(x_Brick) # for increasing accuracy !!!! <== Important!!!!
#x = layers.GlobalAveragePooling2D()(x)
x_Brick = layers.Conv2D(64, 3, padding='same', activation='relu')(x_Brick)
x_Brick = layers.Dropout(0.2)(x_Brick) # for increasing accuracy !!!! <== Important!!!!

x_Brick = layers.GlobalAveragePooling2D()(x_Brick)
x_Brick = layers.Flatten()(x_Brick)
x_Brick = layers.Dense(128, activation='relu')(x_Brick)
output_Brick = layers.Dense(num_classes_Brick, activation = 'softmax')(x_Brick)

model_Brick = tf.keras.Model(inputs_Brick, output_Brick )
#----------------------------------------------------------------#

x_Gray = layers.Conv2D(16, 3, padding='same', activation='relu')(x_Gray)
x_Gray = layers.Dropout(0.1)(x_Gray) # for increasing accuracy !!!! <== Important!!!!

#x = layers.GlobalAveragePooling2D()(x)
print(x_Gray)
x_Gray = layers.Conv2D(32, 3, padding='same', activation='relu')(x_Gray)
x_Gray = layers.Dropout(0.1)(x_Gray) # for increasing accuracy !!!! <== Important!!!!
#x = layers.GlobalAveragePooling2D()(x)
x_Gray = layers.Conv2D(64, 3, padding='same', activation='relu')(x_Gray)
x_Gray = layers.Dropout(0.1)(x_Gray) # for increasing accuracy !!!! <== Important!!!!

x_Gray = layers.GlobalAveragePooling2D()(x_Gray)
x_Gray = layers.Flatten()(x_Gray)
x_Gray = layers.Dense(128, activation='relu')(x_Gray)
output_Gray = layers.Dense(num_classes_Gray, activation = 'softmax')(x_Gray)

model_Gray = tf.keras.Model(inputs_Gray, output_Gray )

print("done")

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 16), dtype=tf.float32, name=None), name='dropout/Identity:0', description="created by layer 'dropout'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 16), dtype=tf.float32, name=None), name='dropout_3/Identity:0', description="created by layer 'dropout_3'")
done


In [14]:
base_learning_rate = 0.001
model_Brick.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_Brick.summary()

#----------------------------------------------------------------#

base_learning_rate = 0.001
model_Gray.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_Gray.summary()

print("done")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 normalization (Normalizatio  (None, 224, 224, 3)      7         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                             

K:\Anaconda\envs\ml\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [15]:
epochs_Brick=1
history_Brick = model_Brick.fit(
  train_ds_Brick,
  validation_data=val_ds_Brick,
  epochs=epochs_Brick
)

#----------------------------------------------------------------#

epochs_Gray=1
history_Gray = model_Gray.fit(
  train_ds_Gray,
  validation_data=val_ds_Gray,
  epochs=epochs_Gray
)

print("done")

K:\Anaconda\envs\ml\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


160/160 [==============================] - 77s 466ms/step - loss: 0.1416 - accuracy: 0.9494 - val_loss: 0.0117 - val_accuracy: 0.9953
done


In [ ]:
acc_Brick = history_Brick.history['accuracy_Brick']
val_acc_Brick = history_Brick.history['val_accuracy_Brick']

loss_Brick = history_Brick.history['loss_Brick']
val_loss_Brick = history_Brick.history['val_loss_Brick']

epochs_range_Brick = range(epochs_Brick)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range_Brick, acc_Brick, label='Training Accuracy_Brick')
plt.plot(epochs_range_Brick, val_acc_Brick, label='Validation Accuracy_Brick')
plt.legend(loc='lower right_Brick')
plt.title('Training and Validation Accuracy_Brick')

plt.subplot(1, 2, 2)
plt.plot(epochs_range_Brick, loss_Brick, label='Training Loss_Brick')
plt.plot(epochs_range_Brick, val_loss_Brick, label='Validation Loss_Brick')
plt.legend(loc='upper right_Brick')
plt.title('Training and Validation Loss_Brick')
plt.show()
#----------------------------------------------------------------#

acc_Gray = history_Gray.history['accuracy']
val_acc_Gray = history_Gray.history['val_accuracy']

loss_Gray = history_Gray.history['loss']
val_loss_Gray = history_Gray.history['val_loss']

epochs_range_Gray = range(epochs_Gray)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range_Gray, acc_Gray, label='Training Accuracy')
plt.plot(epochs_range_Gray, val_acc_Gray, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range_Gray, loss_Gray, label='Training Loss')
plt.plot(epochs_range_Gray, val_loss_Gray, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [16]:
model_Brick.save('lego_classifier_Brick_Mobile_Shape_dropout10.keras')

print('saved_Brick')

#----------------------------------------------------------------#

model_Gray.save('lego_classifier_Gray_Mobile.keras')

print('saved_Gray')

saved_Brick
saved_Gray


In [ ]:
data_augmentation_Brick = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

plt.figure(figsize=(10, 10))
for images_Brick, _ in train_ds_Brick.take(1):
  for i_Brick in range(9):
    augmented_images_Brick = data_augmentation_Brick(images_Brick)
    ax = plt.subplot(3, 3, i_Brick + 1)
    plt.imshow(augmented_images_Brick[0].numpy().astype("uint8"))
    plt.axis("off")

#----------------------------------------------------------------#

data_augmentation_Gray = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

plt.figure(figsize=(10, 10))
for images_Gray, _ in train_ds_Gray.take(1):
  for i_Gray in range(9):
    augmented_images_Gray = data_augmentation_Gray(images_Gray)
    ax_Gray = plt.subplot(3, 3, i_Gray + 1)
    plt.imshow(augmented_images_Gray[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
#----------------------------------------------------------------#


# brick_path = "C:/Users/krzys/Downloads/red.jpg"

# img = tf.keras.utils.load_img(
#     brick_path, target_size=(img_height, img_width)
# )
# img_array = tf.keras.utils.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0) # Create a batch

# predictions = model.predict(img_array)
# score = tf.nn.softmax(predictions[0])

# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(class_names[np.argmax(score)], 100 * np.max(score))
# )



#----------------------------------------------------------------#

In [19]:
import tensorflow as tf
import cv2
import time



cam = cv2.VideoCapture(0)






#model_Brick_model = tf.keras.models.load_model('lego_classifier_Brick_Mobile_Shape_dropout10.keras')
#model_Gray_model = tf.keras.models.load_model('lego_classifier_Gray_Mobile.keras')

model_Brick_model = tf.keras.models.load_model('lego_classifier_Mobile.keras')
model_Gray_model = tf.keras.models.load_model('lego_classifier_Mobile_Shape_dropout10.keras')



test_photo_normal = "C:/Users/krzys/Desktop/frames/frame1.png"
gray_path_save = "K:/LEGOs/Bricks/gray.png"
brick_path = test_photo_normal





while(1):
    
    result, img_Brick = cam.read() #grab frame from webcam
     
    cv2.imwrite(test_photo_normal, img_Brick) #save frame on the test_photo_normal path from cv2 format to png
    
    img_Brick = tf.keras.utils.load_img(
    test_photo_normal, target_size=(img_height, img_width)
    )  #load frame from the test_photo_normal path in tf format
    img_array_Brick = tf.keras.utils.img_to_array(img_Brick)
    img_array_Brick = tf.expand_dims(img_array_Brick, 0) # Create a batch

    predictions_Brick = model_Brick_model.predict(img_array_Brick)
    score_Brick = predictions_Brick[0]

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names_Brick[np.argmax(score_Brick)], 100 * np.max(score_Brick))
    )
    
    
    
    
    
    
    image_gray = cv2.imread(test_photo_normal) #load frame from the same test_photo_normal path in cv2 format
    grayscale = cv2.cvtColor(image_gray, cv2.COLOR_BGR2GRAY) #change into grayscale
    cv2.imwrite(gray_path_save, grayscale) #save grayscale frame on the gray_path_save path
    print("done changing to gray")

    
    
    
    
    
    
    img_Gray = tf.keras.utils.load_img(
        gray_path_save, target_size=(img_height, img_width)
    ) #load grayscale frame from the same test_photo_normal path in tf format
    img_array_Gray = tf.keras.utils.img_to_array(img_Gray)
    img_array_Gray = tf.expand_dims(img_array_Gray, 0) # Create a batch

    predictions_Gray = model_Gray_model.predict(img_array_Gray)
    score_Gray = predictions_Gray[0]

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names_Gray[np.argmax(score_Gray)], 100 * np.max(score_Gray))
        )
    print(
            "-----------------------------------"
            
        )

    time.sleep(0.1)




1/1 [==============================] - 1s 749ms/step
This image most likely belongs to Red_Brick with a 100.00 percent confidence.
done changing to gray
1/1 [==============================] - 1s 568ms/step
This image most likely belongs to 6x1 with a 100.00 percent confidence.
-----------------------------------
1/1 [==============================] - 0s 39ms/step
This image most likely belongs to Red_Brick with a 99.96 percent confidence.
done changing to gray
1/1 [==============================] - 0s 39ms/step
This image most likely belongs to 6x1 with a 99.81 percent confidence.
-----------------------------------
1/1 [==============================] - 0s 28ms/step
This image most likely belongs to Red_Brick with a 99.15 percent confidence.
done changing to gray
1/1 [==============================] - 0s 60ms/step
This image most likely belongs to 6x1 with a 96.39 percent confidence.
-----------------------------------
1/1 [==============================] - 0s 32ms/step
This image mos

KeyboardInterrupt: 

In [ ]:




while(1):
    
    result, image = cam.read()
     
    cv2.imwrite(photo_path, image)
    
    img = tf.keras.utils.load_img(brick_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) 

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))  
    )
    
    

    time.sleep(0.1)